In [ ]:
# Cell 1: 

from transformers import pipeline
import librosa
import tensorflow as tf 

AUDIO_FILE_PATH = "A1-036-KYLE-TOWN-CITY-MALL.mp3" 

print(f"Loading audio file: {AUDIO_FILE_PATH}...")

try:
    input_audio_array, sample_rate = librosa.load(AUDIO_FILE_PATH, sr=16000)
    print("Audio loaded and resampled to 16kHz successfully.")
except Exception as e:
    print(f"Error loading audio file. Make sure '{AUDIO_FILE_PATH}' is in the same directory.")
    print(f"Error: {e}")
    raise


print("\nLoading Whisper ASR model...")

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base"
)

print("Transcribing audio... (This may take a moment)")


transcribed_output = asr_pipeline(input_audio_array, chunk_length_s=30)
transcribed_text = transcribed_output["text"]

print("\n--- ✅ TRANSCRIBED TEXT ---")
print(transcribed_text)


print("\nLoading summarization model...")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary_output = summarizer(
    transcribed_text, 
    max_length=150,
    min_length=30, 
    do_sample=False
)

print("\n--- ✅ FINAL SUMMARY (FROM AUDIO) ---")
print(summary_output[0]['summary_text'])

Loading audio file: A1-036-KYLE-TOWN-CITY-MALL.mp3...
Audio loaded and resampled to 16kHz successfully.

Loading Whisper ASR model...


Device set to use mps:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


Transcribing audio... (This may take a moment)

--- ✅ TRANSCRIBED TEXT ---
 Hello. My name is Kyle. I am from the UK. My question today is, does your town or city have a mall? My town has a big mall. It has many shops for many things. It has cloth stores, shoe stores, department stores. It has a wide range of stores and restaurants. My favourite store is the shoe store because I like to buy shoes. Does your town or city have a mall?

Loading summarization model...


Device set to use mps:0
Your max_length is set to 150, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)



--- ✅ FINAL SUMMARY (FROM AUDIO) ---
My town has a big mall. It has cloth stores, shoe stores, department stores. My favourite store is the shoe store because I like to buy shoes.
